# COVID-19: United States Confirmed Cases & Deaths 

The following data cleaning and merging is performed to load the data into Power BI for further visualization and analysis.

In [1]:
# Packages / libraries
import os 
import numpy as np 
import pandas as pd
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

The data collected for this analysis is operated by the Johns Hopkins University Center for Systems Science and Engineering (https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series)

In [2]:
# load raw data from https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series
confirmed = pd.read_csv("C:/Users/garcr/Desktop/time_series_covid19_confirmed_US.csv")
deaths = pd.read_csv("C:/Users/garcr/Desktop/time_series_covid19_deaths_US.csv")

# Confirm that the dataframes have the same number of rows (countries) and columns (dates)
# Deaths dataframe has an extra column 'Population'
print('The shape of confirmed is:', confirmed.shape)
print('The shape of deaths is:', deaths.shape)

confirmed.head()
#deaths.head()

The shape of confirmed is: (3261, 122)
The shape of deaths is: (3261, 123)


,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,145,145,145,145,149,149,151,151,151,151
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,14,14,14,14,15,15,15,16,16,19
3,630,PR,PRI,630,72.0,NaN,Puerto Rico,US,18.2208,-66.5901,...,1757,1808,1843,1924,1968,2031,2156,2173,2198,2256
4,850,VI,VIR,850,78.0,NaN,Virgin Islands,US,18.3358,-64.8963,...,66,66,66,66,66,66,68,68,69,69


In [3]:
# Drop unnecessary columns
confirmed.drop(['iso2','iso3','code3','FIPS','Combined_Key'], axis=1, inplace=True)
deaths.drop(['iso2','iso3','code3','FIPS','Combined_Key'], axis=1, inplace=True)

#confirmed.head()
#deaths.head()

In [4]:
# melt the date columns into one column
confirmed2 = pd.melt(confirmed, id_vars= ['UID','Admin2','Province_State','Country_Region','Lat','Long_'], var_name=['Date'])
deaths2 = pd.melt(deaths, id_vars= ['UID','Admin2','Province_State','Country_Region','Lat','Long_','Population'], var_name=['Date'])

print('The shape of confirmed is:', confirmed2.shape)
print('The shape of deaths is:', deaths2.shape)

confirmed2.head()

The shape of confirmed is: (361971, 8)
The shape of deaths is: (361971, 9)


,UID,Admin2,Province_State,Country_Region,Lat,Long_,Date,value
0,16,NaN,American Samoa,US,-14.2710,-170.1320,1/22/20,0
1,316,NaN,Guam,US,13.4443,144.7937,1/22/20,0
2,580,NaN,Northern Mariana Islands,US,15.0979,145.6739,1/22/20,0
3,630,NaN,Puerto Rico,US,18.2208,-66.5901,1/22/20,0
4,850,NaN,Virgin Islands,US,18.3358,-64.8963,1/22/20,0


In [5]:
# convert Date column into datetime objects (xxxx-xx-xx)
confirmed2['Date'] = pd.to_datetime(confirmed2['Date'])
deaths2['Date'] = pd.to_datetime(deaths2['Date'])

confirmed2.head()

,UID,Admin2,Province_State,Country_Region,Lat,Long_,Date,value
0,16,NaN,American Samoa,US,-14.2710,-170.1320,2020-01-22,0
1,316,NaN,Guam,US,13.4443,144.7937,2020-01-22,0
2,580,NaN,Northern Mariana Islands,US,15.0979,145.6739,2020-01-22,0
3,630,NaN,Puerto Rico,US,18.2208,-66.5901,2020-01-22,0
4,850,NaN,Virgin Islands,US,18.3358,-64.8963,2020-01-22,0


In [6]:
# Replace column headings with detailed fields
confirmed2.columns = confirmed2.columns.str.replace('value','Confirmed')
confirmed2.columns = confirmed2.columns.str.replace('Admin2','County')
deaths2.columns = deaths2.columns.str.replace('value', 'Deaths')
deaths2.columns = deaths2.columns.str.replace('Admin2','County')

# Investigate NULLs before join
print(confirmed2.isnull().sum())
print(deaths2.isnull().sum())

UID                 0
County            777
Province_State      0
Country_Region      0
Lat                 0
Long_               0
Date                0
Confirmed           0
dtype: int64
UID                 0
County            777
Province_State      0
Country_Region      0
Lat                 0
Long_               0
Population          0
Date                0
Deaths              0
dtype: int64


The null county values are a result of the District of Columbia, and the 5 inhabited US territories (American Samoa, Guam, Northern Mariana Islands, Puerto Rico, US Virgin Islands) not having counties, in addition to the Grand Princess and Diamond Princess cruise ships.

In [7]:
# Join the dataframes 
covid = confirmed2.merge(deaths2[['UID','Province_State','Country_Region','Date','Deaths','Population']],
                      how='outer',
                      left_on=['UID','Province_State','Country_Region','Date'],
                      right_on=['UID','Province_State','Country_Region','Date'])

# Investigate the shape of the dataframe after the join
print('\nThe shape of confirmed is:', confirmed2.shape)
print('The shape of deaths is:', deaths2.shape)
print('The shape of the joined dataframe is:', covid.shape)

#print(covid.isnull().sum())
#covid.tail()


The shape of confirmed is: (361971, 8)
The shape of deaths is: (361971, 9)
The shape of the joined dataframe is: (361971, 10)


The joined dataframe "covid" has 10 columns because Deaths and Population has been added to the confirmed dataframe. 

In [8]:
# Fill County NaN values with Province_State values (Applies only to DC, US territories, and cruise ships)
covid['County'].fillna(covid['Province_State'],inplace=True)

#covid.head()
covid.isnull().sum()

UID               0
County            0
Province_State    0
Country_Region    0
Lat               0
Long_             0
Date              0
Confirmed         0
Deaths            0
Population        0
dtype: int64

In [9]:
# Add Month-Year column
covid['Month-Year'] = covid['Date'].dt.strftime('%b-%Y')
#covid.tail()

In [10]:
# Copy df
temp = covid.copy()

# Create columns for previous date's COVID-19 cases to create daily numbers and the running total 
temp['Temp Date'] = covid['Date'] + pd.Timedelta(days=1)
temp.rename(columns={'Confirmed':'Confirmed - 1', 'Deaths':'Deaths - 1', 'Date':'Date - 1'}, inplace=True)

# Perform left join on DFs
covid_19 = covid.merge(temp[['UID','Province_State','Country_Region','Confirmed - 1','Deaths - 1',
                             'Temp Date','Date - 1']], how='left', 
                       left_on=['UID','Province_State','Country_Region','Date'],
                       right_on=['UID','Province_State','Country_Region','Temp Date'])
print(covid_19.shape)
covid_19.head()

(361971, 15)


,UID,County,Province_State,Country_Region,Lat,Long_,Date,Confirmed,Deaths,Population,Month-Year,Confirmed - 1,Deaths - 1,Temp Date,Date - 1
0,16,American Samoa,American Samoa,US,-14.2710,-170.1320,2020-01-22,0,0,55641,Jan-2020,NaN,NaN,NaT,NaT
1,316,Guam,Guam,US,13.4443,144.7937,2020-01-22,0,0,164229,Jan-2020,NaN,NaN,NaT,NaT
2,580,Northern Mariana Islands,Northern Mariana Islands,US,15.0979,145.6739,2020-01-22,0,0,55144,Jan-2020,NaN,NaN,NaT,NaT
3,630,Puerto Rico,Puerto Rico,US,18.2208,-66.5901,2020-01-22,0,0,2933408,Jan-2020,NaN,NaN,NaT,NaT
4,850,Virgin Islands,Virgin Islands,US,18.3358,-64.8963,2020-01-22,0,0,107268,Jan-2020,NaN,NaN,NaT,NaT


In [11]:
# Calculate the daily numbers for confirmed cases & deaths (current aggregate - previous aggregate = daily confirmed)
covid_19['Daily Confirmed'] = covid_19['Confirmed'] - covid_19['Confirmed - 1']
covid_19['Daily Deaths'] = covid_19['Deaths'] - covid_19['Deaths - 1']

print(covid_19.shape)
covid_19.head()

(361971, 17)


,UID,County,Province_State,Country_Region,Lat,Long_,Date,Confirmed,Deaths,Population,Month-Year,Confirmed - 1,Deaths - 1,Temp Date,Date - 1,Daily Confirmed,Daily Deaths
0,16,American Samoa,American Samoa,US,-14.2710,-170.1320,2020-01-22,0,0,55641,Jan-2020,NaN,NaN,NaT,NaT,NaN,NaN
1,316,Guam,Guam,US,13.4443,144.7937,2020-01-22,0,0,164229,Jan-2020,NaN,NaN,NaT,NaT,NaN,NaN
2,580,Northern Mariana Islands,Northern Mariana Islands,US,15.0979,145.6739,2020-01-22,0,0,55144,Jan-2020,NaN,NaN,NaT,NaT,NaN,NaN
3,630,Puerto Rico,Puerto Rico,US,18.2208,-66.5901,2020-01-22,0,0,2933408,Jan-2020,NaN,NaN,NaT,NaT,NaN,NaN
4,850,Virgin Islands,Virgin Islands,US,18.3358,-64.8963,2020-01-22,0,0,107268,Jan-2020,NaN,NaN,NaT,NaT,NaN,NaN


In [12]:
# Include daily numbers for the first day of data where there is no previous date available (2020-01-22)
covid_19['Daily Confirmed'].loc[covid_19['Date'] == '2020-01-22'] = covid_19['Confirmed']
covid_19['Daily Deaths'].loc[covid_19['Date'] == '2020-01-22'] = covid_19['Deaths']

# Delete unnecessary columns
del covid_19['Confirmed - 1'] 
del covid_19['Deaths - 1']
del covid_19['Temp Date']
del covid_19['Date - 1']

covid_19.head()

,UID,County,Province_State,Country_Region,Lat,Long_,Date,Confirmed,Deaths,Population,Month-Year,Daily Confirmed,Daily Deaths
0,16,American Samoa,American Samoa,US,-14.2710,-170.1320,2020-01-22,0,0,55641,Jan-2020,0.0,0.0
1,316,Guam,Guam,US,13.4443,144.7937,2020-01-22,0,0,164229,Jan-2020,0.0,0.0
2,580,Northern Mariana Islands,Northern Mariana Islands,US,15.0979,145.6739,2020-01-22,0,0,55144,Jan-2020,0.0,0.0
3,630,Puerto Rico,Puerto Rico,US,18.2208,-66.5901,2020-01-22,0,0,2933408,Jan-2020,0.0,0.0
4,850,Virgin Islands,Virgin Islands,US,18.3358,-64.8963,2020-01-22,0,0,107268,Jan-2020,0.0,0.0


In [13]:
# checking most recent US daily totals
daily_sum = covid_19.groupby('Date').agg({'Daily Confirmed': 'sum'})
print(daily_sum.tail())
daily_deaths = covid_19.groupby('Date').agg({'Daily Deaths': 'sum'})
print(daily_deaths.tail())

            Daily Confirmed
Date                       
2020-05-07          27692.0
2020-05-08          26906.0
2020-05-09          25621.0
2020-05-10          19710.0
2020-05-11          18621.0
            Daily Deaths
Date                    
2020-05-07        2207.0
2020-05-08        1518.0
2020-05-09        1615.0
2020-05-10         731.0
2020-05-11        1156.0


In [14]:
# Export Data as csv to load into Power BI
covid_19.to_csv('UScovid', sep='\t')